# <center> Forces <center>
Este notebook explica cómo utilizamos la cinemática obtenida para resolver la dinámica.

Consideramos que existen 3 fuerzas:
- Bicep
- Peso pesa
- Peso antebrazo

**Forces** se encarga de:
- Calcular momentos.
- Obtener la fuerza del bicep.
- Obtener componentes de vectores para graficarlos en el video.
- Calcular el trabajo realizado por el bicep.


#### Inicializaciones

In [1]:
import numpy as np
from video import Video
from data import Data
from adapter import Adapter
from unit_converter import Unit_converter
import column_filter as cf
import plotly.express as px
from storage import get_input_video, get_keypoints

DISTANCE_ELBOW_WRIST =  0.3
MASS_WEIGHT = 1
MASS_FOREARM = 1
RADIUS_BICEP = 0.04
VIDEO_NAME = "video2"
EXTENSION = "mp4"

video = get_input_video(VIDEO_NAME, EXTENSION)
objects = ["shoulder", "elbow", "wrist"]
positions = get_keypoints(VIDEO_NAME)

adapter = Adapter(positions, objects, video.get_height())
data = Data(adapter.get_adapted_data(), objects)
df = data.get_data()
raw_data = df[:]

unit_converter = Unit_converter(df['r_wrist'].iloc[0], DISTANCE_ELBOW_WRIST, video.get_fps(), 1)
df = unit_converter.convert_position(df, cf.position_columns(df.columns))
df = unit_converter.convert_velocity(df, cf.velocity_columns(df.columns))
df = unit_converter.convert_acceleration(df, cf.acceleration_columns(df.columns))
df = unit_converter.convert_angular_velocity(df, ['angular_velocity'])
df = unit_converter.convert_angular_acceleration(df, ['angular_acceleration'])
df = unit_converter.convert_time(df, ['time'])
df = unit_converter.convert_position(df, ['distance_elbow_shoulder', 'x_vector_elbow_to_shoulder', 'y_vector_elbow_to_shoulder'])

g = -9.81

#### Cálculo de momentos
Ya conocemos el radio del codo a la pesa, a partir de este obtenemos el del codo al medio del antebrazo

In [2]:
radius_weight = df['r_wrist']
radius_forearm = radius_weight / 2

Con estos radios podemos calcular los momentos de inercia utilizando $I=mr^2$

In [3]:
inertia_weight = MASS_WEIGHT * radius_weight ** 2
inertia_forearm = MASS_FOREARM * radius_forearm ** 2

Ahora usando $M=I\dot\omega$ podemos obtener la suma de los momentos.

In [4]:
sum_moment = (inertia_weight + inertia_forearm) * df['angular_acceleration']
fig = px.line(x = df['time'], y = sum_moment)
fig.show()

El momento de una fuerza es $\vec{M}=\vec{r}\times\vec{F}$, entonces $|\vec{M}|=|\vec{r}||\vec{F}|\sin\alpha$, donde $\alpha$ es el ángulo entre $\vec{r}$ y $\vec{F}$.
Obtenemos $\alpha$ a partir de $\theta$.

In [5]:
angle_forearm_g = np.pi - df['theta_wrist']

fig = px.line(x = df['time'], y = angle_forearm_g)
fig.show()

Entonces obtenemos los momentos de los pesos.

In [6]:
moment_weight = radius_weight * MASS_WEIGHT * g * np.sin(angle_forearm_g)
moment_forearm = radius_forearm * MASS_FOREARM * g * np.sin(angle_forearm_g)

fig = px.line(x = df['time'], y = [moment_weight, moment_forearm])
fig.show()

#### Cálculo de la fuerza del bicep
Sabemos que $\vec{M}=\vec{M_w}+\vec{M_f}+\vec{M_b}$ y ya conocemos $\vec{M},\vec{M_w},\vec{M_f}$ por lo que despejando podemos obtener $\vec{M_b}$ y utilizarlo para calcular $\vec{F_b}$

In [7]:
df['force_bicep'] = (sum_moment - moment_weight - moment_forearm) / (RADIUS_BICEP * np.sin(df['theta_wrist']))

fig = px.line(x = df['time'], y = df['force_bicep'])
fig.show()

#### Obtener componentes de vectores para graficarlos en el video
Podemos obtener fácilmente los componentes de los pesos, $x=0$, $y=mg$

In [8]:
df['px_weight'] = 0
df['py_weight'] = MASS_WEIGHT * g

df['px_forearm'] = 0
df['py_forearm'] = MASS_FOREARM * g

Para la fuerza del bicep, primero obtenemos su origen.

Conocemos la posición del codo y su distancia al bicep, para obtener la posición del bicep necesitamos la distancia en $x$ y la distancia en $y$.
Podemos obtenerlas porque conocemos el ángulo del antebrazo con la horizontal y con la vertical.

In [9]:
df['rx_bicep'] = df['rx_elbow'] + RADIUS_BICEP * np.cos(df['angle_forearm_horizontal'])
df['ry_bicep'] = df['ry_elbow'] + RADIUS_BICEP * np.cos(df['angle_forearm_vertical'])

Para las componentes del vector, utilizamos su módulo y el versor del codo al hombro.

In [10]:
x_versor_elbow_to_shoulder = df['x_vector_elbow_to_shoulder'] / df['distance_elbow_shoulder']
y_versor_elbow_to_shoulder = df['y_vector_elbow_to_shoulder'] / df['distance_elbow_shoulder']

df['fx_bicep'] = df['force_bicep'] * x_versor_elbow_to_shoulder
df['fy_bicep'] = df['force_bicep'] * y_versor_elbow_to_shoulder

#### Cálculo del trabajo
$W=\int\vec{F}\vec{d}$ entonces nosotros calculamos $\sum|\vec{F}||\vec{d}|$ donde $d$ es la distancia y la fuerza es en la dirección del desplazamiento.

Primero obtenemos la fuerza en la dirección del desplazamiento

In [11]:
angle_bicep_movement = df['theta_wrist'] + np.pi / 2 * df['angular_velocity'] / np.abs(df['angular_velocity'])
projected_force = df['force_bicep'] * np.cos(angle_bicep_movement)

Luego utilizamos el teorema del coseno para calcular la longitud del bicep $a^2=b^2+c^2-2bc\cos\alpha$

In [12]:
distance = np.sqrt(df['distance_elbow_shoulder'] ** 2 + RADIUS_BICEP ** 2 -
                           2 * df['distance_elbow_shoulder'] * RADIUS_BICEP *
                           np.cos(df['theta_wrist']))
delta_distance = distance.diff()

Calculamos el trabajo total sumando los valores absolutos

In [13]:
work_i = projected_force * np.abs(delta_distance)
work_i_abs = np.abs(work_i)

print(f"work: {np.sum(work_i_abs)}")

work: 53.83702976464163
